In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import graycomatrix, graycoprops
from skimage import data
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array,array_to_img, load_img
import shutil
import os
import cv2
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Activation,Conv1D,MaxPooling1D,BatchNormalization, GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score,classification_report
from tensorflow.keras.layers import Dropout,Input
from tensorflow.keras import regularizers
from tqdm import tqdm
import skimage

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

datagen = ImageDataGenerator(
    rotation_range=4,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    brightness_range=(0.8, 1.2),  # Adjust brightness
    rescale=1./255,  # Rescale pixel values
    channel_shift_range=20.0,  # Shift color channels
    zca_whitening=True  # Apply ZCA whitening
)

img = load_img(r"C:\Users\PC\Desktop\PROJECT\source6\AK (8).jpg")
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in datagen.flow(x, batch_size=1, save_to_dir="view", save_prefix="pet", save_format="jpg"):
    i += 1
    if i > 20:
        break


C:\Users\PC\anaconda3\envs\tf-origin\lib\site-packages\keras\src\legacy\preprocessing\image.py:1047: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn(
C:\Users\PC\anaconda3\envs\tf-origin\lib\site-packages\keras\src\legacy\preprocessing\image.py:1263: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
C:\Users\PC\anaconda3\envs\tf-origin\lib\site-packages\keras\src\legacy\preprocessing\image.py:1286: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


In [3]:
def apply_gabor_filter(image, ksize, sigma, theta, lambd, gamma):
    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, 0, ktype=cv2.CV_32F)
    filtered_image = cv2.filter2D(image,cv2.CV_32F, kernel)
    return filtered_image

def extract_glcms(image):
    image = np.uint8(image)
    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    glcm = graycomatrix(image, distances=distances, angles=angles, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast').ravel()
    dissimilarity = graycoprops(glcm, 'dissimilarity').ravel()
    homogeneity = graycoprops(glcm, 'homogeneity').ravel()
    energy = graycoprops(glcm, 'energy').ravel()
    correlation = graycoprops(glcm, 'correlation').ravel()
    return np.concatenate([contrast, dissimilarity, homogeneity, energy, correlation])

In [4]:
def horizontal_projection_profile(image):
    hpp = np.sum(image, axis=1)
    return hpp

def word_line_space_formation(hpp):
    threshold = np.mean(hpp) * 0.1
    space_indices = np.where(hpp < threshold)[0]
    word_line_spaces = np.diff(space_indices)
    return word_line_spaces

def image_padding(image, target_size=(650, 650)):
    h, w, c = image.shape
    padded_image = np.zeros((target_size[0], target_size[1], c))
    padded_image[:min(h, target_size[0]), :min(w, target_size[1])] = image[:min(h, target_size[0]), :min(w, target_size[1])]
    return padded_image

def block_normalization(image, block_size=(650, 650)):
    normalized_blocks = []
    for i in range(0, image.shape[0], block_size[0]):
        for j in range(0, image.shape[1], block_size[1]):
            block = image[i:i+block_size[0], j:j+block_size[1]]
            block_mean = np.mean(block)
            block_std = np.std(block)
            normalized_block = (block - block_mean) / (block_std + 1e-5)
            normalized_blocks.append(normalized_block)
    return np.array(normalized_blocks)

In [5]:
from sklearn.preprocessing import MinMaxScaler

def extract_features():
    features = []
    labels = []
    datadir = r"C:\Users\PC\Desktop\PROJECT\HANDWRITING"
    categories = ["AMBANI", "BLESSING", "CELESTINA","ENOCH", "HAKEEM", "PETER", "RIDWAN", "TOBA"]
    for category in tqdm(categories):
        path = os.path.join(datadir, category)
        image_names = os.listdir(path)
        for img_name in (image_names):
            img_path = os.path.join(path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img_array = np.expand_dims(img, axis=-1)
            img_size = 850
            new_array = cv2.resize(img_array, (img_size, img_size))
            img_gray = cv2.adaptiveThreshold(new_array, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5, 11)
            bilateral_filtered = cv2.bilateralFilter(img_gray, 5, 6, 6)
            gaussian_blur = cv2.GaussianBlur(bilateral_filtered, (7, 7), 2)
            new_image = cv2.addWeighted(bilateral_filtered, 1.5, gaussian_blur, -0.5, 0)
            ksize = 30
            sigma = 0.2
            theta = 0
            lambd = 10
            gamma = 0.5
            gabor_filtered = apply_gabor_filter(new_image, ksize, sigma, theta, lambd, gamma)
            # new_image = cv2.resize(gabor_filtered, (850, 850))
            gabor_filtered = np.expand_dims(gabor_filtered, axis=-1)
            hpp = horizontal_projection_profile(gabor_filtered)
            word_line_spaces = word_line_space_formation(hpp)
            padded_image = image_padding(gabor_filtered, target_size=(650, 650))
            normalized_blocks = block_normalization(padded_image, block_size=(650, 650))
            reshaped_blocks = normalized_blocks.reshape(normalized_blocks.shape[1], normalized_blocks.shape[2], normalized_blocks.shape[0]).squeeze()
            
            
            glcm_features = extract_glcms(reshaped_blocks)
            features.append(glcm_features)
            labels.append(category)  
            # new_img_path = os.path.join(destination_folder,img_name)
            # cv2.imwrite(new_img_path, gabor_filtered)
            # plt.imshow(reshaped_blocks, cmap="gray")
            # plt.show()
    features = np.array(features)
    labels = np.array(labels)

    
    scaler = MinMaxScaler()
    features_normalized = scaler.fit_transform(features)
    
    return features_normalized, labels, scaler
features, labels, scaler  = extract_features()
print("Shape of features:", features.shape)
print("Shape of labels:", labels.shape)
print("Labels:", labels)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:46<00:00, 13.33s/it]

Shape of features: (1297, 20)
Shape of labels: (1297,)
Labels: ['AMBANI' 'AMBANI' 'AMBANI' ... 'TOBA' 'TOBA' 'TOBA']


In [6]:
features

array([[0.38039455, 0.45977794, 0.49537217, ..., 0.34270042, 0.32684579,
        0.37820802],
       [0.42169531, 0.44421932, 0.46413582, ..., 0.43410331, 0.43100178,
        0.44344155],
       [0.34753023, 0.40569835, 0.442048  , ..., 0.38673364, 0.35417323,
        0.36803723],
       ...,
       [0.36558672, 0.37993424, 0.42813865, ..., 0.40096928, 0.3340799 ,
        0.31917631],
       [0.25044952, 0.32960669, 0.39575376, ..., 0.32983445, 0.21297736,
        0.26566279],
       [0.18656346, 0.28284959, 0.33476349, ..., 0.40802858, 0.3044716 ,
        0.36354297]])

In [7]:
labels

array(['AMBANI', 'AMBANI', 'AMBANI', ..., 'TOBA', 'TOBA', 'TOBA'],
      dtype='<U9')

In [12]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)


onehot_encoder = OneHotEncoder()
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
one_hot_encoded = onehot_encoder.fit_transform(integer_encoded)


labels = one_hot_encoded.toarray()

In [13]:
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [9]:
labels.shape

(1297, 8)

In [10]:
features.shape

(1297, 20)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [12]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (1037, 20)
Shape of X_test: (260, 20)
Shape of y_train: (1037, 8)
Shape of y_test: (260, 8)


In [13]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

Shape of X_train: (1037, 20, 1)
Shape of X_test: (260, 20, 1)


In [14]:
from keras.layers import Input

model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    Conv1D(32, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(2),
    Conv1D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(2),
    GlobalAveragePooling1D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(8, activation='softmax')
])

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 20, 32)              │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 20, 32)              │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 10, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 10, 64)              │           6,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 10, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 5, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 5, 128)              │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 5, 128)              │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 2, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 2, 128)              │          49,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 2, 128)              │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 1, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 8)                   │           2,056 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 281,160 (1.07 MB)

 Trainable params: 280,456 (1.07 MB)

 Non-trainable params: 704 (2.75 KB)

In [16]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
model.fit(X_train, y_train, 
batch_size=32, 
epochs=40, 
validation_split=0.2,
verbose=1)

Epoch 1/40
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.2033 - loss: 2.2721 - val_accuracy: 0.1683 - val_loss: 2.0844
Epoch 2/40
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.4393 - loss: 1.5622 - val_accuracy: 0.1683 - val_loss: 2.0965
Epoch 3/40
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.5617 - loss: 1.2669 - val_accuracy: 0.1683 - val_loss: 2.1248
Epoch 4/40
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.6280 - loss: 1.0858 - val_accuracy: 0.1683 - val_loss: 2.1687
Epoch 5/40
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6827 - loss: 0.9264 - val_accuracy: 0.1587 - val_loss: 2.2335
Epoch 6/40
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7295 - loss: 0.7989 - val_accuracy: 0.1202 - val_loss: 2.3087
Epoch 7/40
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7652 - loss: 0.7169 - val_accuracy: 0.1010 - val_loss: 2.3744
Epoch 8/40
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8029 - loss: 0.5910 - val_accuracy: 0.1250 - 

In [18]:
model.evaluate(X_test,y_test)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9411 - loss: 0.1324


[0.14323320984840393, 0.9384615421295166]

In [19]:
y_pred = model.predict(X_test)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step


In [20]:
y_pred_classes = [np.argmax(element) for element in y_pred]
y_test_classes = np.argmax(y_test, axis=1)

In [21]:
print(classification_report(y_test_classes,y_pred_classes))

              precision    recall  f1-score   support

           0       0.86      0.96      0.91        26
           1       0.95      1.00      0.97        18
           2       0.88      0.97      0.92        29
           3       1.00      0.94      0.97        35
           4       1.00      0.96      0.98        25
           5       0.90      0.97      0.94        39
           6       0.98      0.98      0.98        54
           7       0.93      0.74      0.82        34

    accuracy                           0.94       260
   macro avg       0.94      0.94      0.94       260
weighted avg       0.94      0.94      0.94       260



In [22]:
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img_array = np.expand_dims(img, axis=-1)
    img_size = 850
    new_array = cv2.resize(img_array, (img_size, img_size))
    img_gray = cv2.adaptiveThreshold(new_array, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5, 11)
    bilateral_filtered = cv2.bilateralFilter(img_gray, 5, 6, 6)
    gaussian_blur = cv2.GaussianBlur(bilateral_filtered, (7, 7), 2)
    new_image = cv2.addWeighted(bilateral_filtered, 1.5, gaussian_blur, -0.5, 0)
    ksize = 30
    sigma = 0.2
    theta = 0
    lambd = 10
    gamma = 0.5
    gabor_filtered = apply_gabor_filter(new_image, ksize, sigma, theta, lambd, gamma)
    gabor_filtered = np.expand_dims(gabor_filtered, axis=-1)
    hpp = horizontal_projection_profile(gabor_filtered)
    word_line_spaces = word_line_space_formation(hpp)
    padded_image = image_padding(gabor_filtered, target_size=(650, 650))
    normalized_blocks = block_normalization(padded_image, block_size=(650, 650))
    reshaped_blocks = normalized_blocks.reshape(normalized_blocks.shape[1], normalized_blocks.shape[2], normalized_blocks.shape[0]).squeeze()
    glcm_features = extract_glcms(reshaped_blocks)
    return glcm_features

In [23]:
def make_prediction(img_path, model, scaler):
    features = preprocess_image(img_path)
    features_normalized = scaler.transform([features])
    prediction = model.predict(features_normalized.reshape(1,20,1))
    predicted_class = np.argmax(prediction)
    return predicted_class

In [25]:
def make_prediction_with_confidence(img_path, model, scaler, label_encoder, threshold=0.4):
    
    features = preprocess_image(img_path)
    features_normalized = scaler.transform([features])
    
    
    prediction = model.predict(features_normalized.reshape(1,-1, 1))
    predicted_class = np.argmax(prediction)
    confidence = prediction[0][predicted_class]
    
    
    if confidence < threshold:
        return "unrecognized", confidence
    else:
        
        predicted_label = label_encoder.inverse_transform([predicted_class])[0]
        return predicted_label, confidence

img_path = r"C:\Users\PC\Desktop\PROJECT\HANDWRITING\PETER\pete (52).jpg"
analysis, confidence = make_prediction_with_confidence(img_path, model, scaler, label_encoder, threshold=0.4)

print(f"Prediction: {analysis}, Confidence: {confidence:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Prediction: PETER, Confidence: 1.00


In [25]:
img_path = r"C:\Users\PC\Desktop\PROJECT\test\CELESTINA\celes (10).jpg"
# features_normalized, labels, scaler  = extract_features()  
# scaler.transform(features_normalized)  
analysis = make_prediction(img_path, model, scaler)
print("Prediction:", analysis)

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - src data type = object is not supported
>  - Expected Ptr<cv::UMat> for argument 'src'


In [ ]:
analysis = label_encoder.inverse_transform([analysis])[0]
print("Prediction:", analysis)

In [ ]:
labels.shape

In [ ]:
analysis

In [ ]:
import joblib
model.save("handwriting_recognition_model.keras")

with open("label_encoder.pkl", "wb") as le_file:
    
    joblib.dump(label_encoder, le_file)
with open("scaler.pkl", "wb") as scaler_file:
    joblib.dump(scaler, scaler_file)
print("saved successfully.")

In [81]:
"C:\Users\PC\Desktop\PROJECT\unrecognized\unrecognized (6).jpg"

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2920531907.py, line 1)